In [1]:
import random
import pandas as pd

def simulate_damage(base_damage, percent_damage_mod, crit_chance, crit_damage, num_hits, use_ok, ok_proc_chance, collector_proc_chance, collector_multiplier):
    hits = []
    proc_data = []

    for _ in range(num_hits):
        # Calculate the regular damage with modifiers
        regular_damage = base_damage * (1 + percent_damage_mod / 100)

        is_crit = False
        proc_occurred = False
        proc_depth = 0
        collector_proc_occurred = False

        # Determine if the hit is a critical hit
        if random.random() < crit_chance / 100:
            is_crit = True
            crit_extra_damage = regular_damage * (crit_damage / 100)
            total_damage = regular_damage + crit_extra_damage
            
            if use_ok:
              # Handle Overkill chain calculation
              multiplier = 1
              while random.random() < ok_proc_chance / 100:
                  proc_occurred = True
                  proc_depth += 1
                  multiplier *= 2
                  if multiplier >= 1000:
                      break
              crit_extra_damage *= multiplier
              damage = regular_damage + crit_extra_damage

            # Handle the collector proc (only on crit)
            if random.random() < collector_proc_chance / 100:
                collector_proc_occurred = True
                damage *= collector_multiplier
        else:
            damage = regular_damage

        hits.append(damage)
        proc_data.append({
            'Damage': damage,
            'IsCrit': is_crit,
            'OKProcOccurred': proc_occurred,
            'OKProcDepth': proc_depth,
            'CollectorProcOccurred': collector_proc_occurred
        })

    return hits, proc_data

# Example usage
base_damage = 10
percent_damage_mod = 79  # 20% additional damage
crit_chance = 49  # 25% chance to crit
crit_damage = 234  # 50% additional damage on crit
num_hits = 100000
ok_proc_chance = 24  # 10% chance to proc on crit
collector_proc_chance = 15  # 15% chance to multiply the entire hit on crit
collector_multiplier = 3  # Multiplier for the entire hit

hits, proc_data = simulate_damage(base_damage, percent_damage_mod, crit_chance, crit_damage, num_hits, True, ok_proc_chance, collector_proc_chance, collector_multiplier)

# Convert hits and proc data to a pandas DataFrame for analysis
df_hits = pd.DataFrame(hits, columns=['Damage'])
df_proc_data = pd.DataFrame(proc_data)

In [2]:
# Convert hits and proc data to a pandas DataFrame for analysis
df_hits = pd.DataFrame(hits, columns=['Damage'])
df_proc_data = pd.DataFrame(proc_data)

# Basic statistics
print(df_hits.describe())
print("\n")
print(df_proc_data.value_counts())

              Damage
count  100000.000000
mean       59.934205
std       115.042196
min        17.900000
25%        17.900000
50%        17.900000
75%        59.786000
max     16137.924000


Damage     IsCrit  OKProcOccurred  OKProcDepth  CollectorProcOccurred
17.900     False   False           0            False                    50884
59.786     True    False           0            False                    31831
101.672    True    True            1            False                     7543
179.358    True    False           0            True                      5487
185.444    True    True            2            False                     1836
305.016    True    True            1            True                      1416
352.988    True    True            3            False                      445
556.332    True    True            2            True                       313
688.076    True    True            4            False                       96
1058.964   True    True     